In [84]:
## Python decorators are example of dp creeping into the language itself!
import time

def time_it(func):
  def wrapper():
    start = time.time()
    result = func()
    end = time.time()
    print(f'{func.__name__} took {int((end-start)*1000)}ms')
  return wrapper

@time_it
def some_op():
  print('Starting op')
  time.sleep(1)
  print('We are done')
  return 123

if __name__ == '__main__':
  # some_op()
  # time_it(some_op)()
  some_op()


Starting op
We are done
some_op took 1002ms


In [85]:
## classic decorator class
from abc import ABC


class Shape(ABC):
    def __str__(self):
        return ''


class Circle(Shape):
    def __init__(self, radius=0.0):
        self.radius = radius

    def resize(self, factor):
        self.radius *= factor

    def __str__(self):
        return f'A circle of radius {self.radius}'


class Square(Shape):
    def __init__(self, side):
        self.side = side

    def __str__(self):
        return f'A square with side {self.side}'


class ColoredShape(Shape):
    def __init__(self, shape, color):
        if isinstance(shape, ColoredShape):
            raise Exception('Cannot apply ColoredDecorator twice')
        self.shape = shape
        self.color = color

    def __str__(self):
        return f'{self.shape} has the color {self.color}'


class TransparentShape(Shape):
    def __init__(self, shape, transparency):
        self.shape = shape
        self.transparency = transparency

    def __str__(self):
        return f'{self.shape} has {self.transparency * 100.0}% transparency'


if __name__ == '__main__':
    circle = Circle(2)
    print(circle)

    red_circle = ColoredShape(circle, "red")
    print(red_circle)

    # ColoredShape doesn't have resize()
    # red_circle.resize(3)

    red_half_transparent_square = TransparentShape(red_circle, 0.5)
    print(red_half_transparent_square)

    # nothing prevents double application
    mixed = ColoredShape(ColoredShape(Circle(3), 'red'), 'blue')
    print(mixed)

    

A circle of radius 2
A circle of radius 2 has the color red
A circle of radius 2 has the color red has 50.0% transparency


Exception: Cannot apply ColoredDecorator twice

In [86]:
red_circle = ColoredShape(circle, "red")
print(red_circle)

A circle of radius 2 has the color red


In [87]:
## Even though the decorated circle is related to the circle class it does not 
## inherit all its functionality and that is a big drawback of decorators. you do not hace
## access to all the underlying functionality of the baseclasss. This is address in dynamic decorators next.

red_circle.resize(3)

AttributeError: 'ColoredShape' object has no attribute 'resize'

In [88]:
## -hasattr() check if a class has some attribute
# getattr(), find the value of a specified attribute of a class 
## setattr(), define a new attribute of a class 
## delattr(), delete some attribute of a class
## issubclass(A,B) is A a subclass of B ?
## Getters, Setters and deleters of properties 
class Employee:

    def __init__(self, first, last):
        self.first = first
        self.last = last

    @property
    def email(self):
        return '{}.{}@email.com'.format(self.first, self.last)

    @property
    def fullname(self):
        return '{} {}'.format(self.first, self.last)
    
    @fullname.setter
    def fullname(self, name):
        first, last = name.split(' ')
        self.first = first
        self.last = last
    
    @fullname.deleter
    def fullname(self):
        print('Delete Name!')
        self.first = None
        self.last = None


emp_1 = Employee('John', 'Smith')
emp_1.fullname = "Corey Schafer"

print(emp_1.first)
print(emp_1.email)
print(emp_1.fullname)

del emp_1.fullname

Corey
Corey.Schafer@email.com
Corey Schafer
Delete Name!


In [90]:
 emp_1.fullname

'None None'

In [91]:
class SomeObject:
    pass
obj=SomeObject()
obj.x=5
print(obj.x)
del obj.x
print(obj.x)


5


AttributeError: 'SomeObject' object has no attribute 'x'

In [92]:
setattr(obj,'x',10)
print(obj.x)

10


In [ ]:
print(getattr(obj,'x'))

In [ ]:
del obj.x
print(getattr(obj,'x',1000))

In [ ]:
print(obj.x)

In [ ]:
setattr(obj,'x',10)
obj.x

In [ ]:
delattr(obj,'x')
obj.x

In [118]:
## Dynamic Decorators
## FileWithLoggingclass does not inherit from a file class. We can however proxy or forward calls to the 
## underlying file object. Otherwise we mayhave to copy all the underlying object attributes in the decorator class

class FileWithLogging:
  def __init__(self, file):
    self.file = file
  
  def writelines(self, strings):
    self.file.writelines(strings)
    print(f'wrote {len(strings)} lines')

  def __iter__(self):
    return self.file.__iter__()

  def __next__(self):
    return self.file.__next__()

  def __getattr__(self, item):
    return getattr(self.__dict__['file'], item)

  def __setattr__(self, key, value):
    if key == 'file':
      self.__dict__[key] = value
    else:
      setattr(self.__dict__['file'], key,value)

  def __delattr__(self, item):
    delattr(self.__dict__['file'], item)


if __name__ == '__main__':
  file = FileWithLogging(open('hello.txt', 'w'))
  file.writelines(['hello', 'world'])
  file.write('testing')
  file.close()


wrote 2 lines


In [112]:
class Car(list):
    def __init__(self,make,model, year):
        self.make=make
        self.model=model
        self.year=year
        
    def honk(self):
        print("beeb beep!")
        
    def __iter__(self):
        yield self
    
        
        
class TransportTruck:
  def __init__(self, car):
    self.car=car
    
    
  
  def honk(self):
    self.car.honk()
    print('transporting car honks')

  def __iter__(self):
    return self.car.__iter__()

  def __getattr__(self, item):
    return getattr(self.__dict__['car'], item)

  def __setattr__(self, key, value):
    if key == 'car':
      self.__dict__[key] = value
    else:
      setattr(self.__dict__['car'], key,value)

#   def __delattr__(self, item):
#     delattr(self.__dict__['file'], item)


In [113]:
toyota=Car("Toyotta","Camry",2016)
truck=TransportTruck(toyota)
truck.honk()

beeb beep!
transporting car honks


In [114]:
for g in toyota:
    print(g.make)

Toyotta


In [108]:
for g in truck:
    print(g.make)

Toyotta


In [98]:
toyota.model

'Camry'

In [99]:
truck.model

'Camry'

In [100]:
truck.car.width=10


In [101]:
truck.car.width

10

In [102]:
truck.__dict__

{'car': [[...]]}

In [115]:
class wheel:
    pass
w=wheel()
truck.w="axel"
    

In [116]:
truck.__dict__

{'car': [[...]]}

In [117]:
truck.car.__dict__

{'make': 'Toyotta', 'model': 'Camry', 'w': 'axel', 'year': 2016}

In [121]:
from unittest import TestCase

class Circle:
  def __init__(self, radius):
    self.radius = radius

  def resize(self, factor):
    self.radius *= factor

  def __str__(self):
    return 'A circle of radius %s' % self.radius

class Square:
  def __init__(self, side):
    self.side = side

  def __str__(self):
    return 'A square with side %s' % self.side


class ColoredShape:
  def __init__(self, shape, color):
    self.color = color
    self.shape = shape
# callable() in Python. In general, a callable is something that can be called. This built-in method 
# in Python checks and returns True if the object passed appears to be callable, but may not be, 
# otherwise False.
  def resize(self, factor):
    r = getattr(self.shape, 'resize', None)
    if callable(r):
      self.shape.resize(factor)

  def __str__(self):
    return "%s has the color %s" %\
           (self.shape, self.color)

class Evaluate(TestCase):
  def test_circle(self):
    circle = ColoredShape(Circle(5), 'red')
    self.assertEqual(
      'A circle of radius 5 has the color red',
      str(circle)
    )
    circle.resize(2)
    self.assertEqual(
      'A circle of radius 10 has the color red',
      str(circle)
    )

  def test_no_resize_in_square(self):
    square = Square(4)
    r = getattr(square, 'resize', None)
    self.assertFalse(callable(r),
                     'Please do not add resize() to Square')

  def test_square(self):
    square = ColoredShape(Square(2), 'blue')
    self.assertEqual(
      'A square with side 2 has the color blue',
      str(square)
    )
    square.resize(2)
    self.assertEqual(
      'A square with side 2 has the color blue',
      str(square)
    )